In [1]:
from d2l import torch as d2l
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Inception(nn.Module):
    def __init__(self, num_inputs, c1, c2, c3, c4):
        super().__init__()
        self.num_inputs = num_inputs
        self.c1 = c1
        self.c2 = c2
        self.c3 = c3
        self.c4 = c4
        
        self.p1 = nn.Conv2d(self.num_inputs, self.c1, kernel_size=1)
        self.p2 = nn.Sequential(
            nn.Conv2d(self.num_inputs, self.c2[0], kernel_size=1), nn.ReLU(),
            nn.Conv2d(self.c2[0], self.c2[1], kernel_size=3, padding=1), nn.ReLU()
        )
        self.p3 = nn.Sequential(
            nn.Conv2d(self.num_inputs, self.c3[0], kernel_size=1), nn.ReLU(),
            nn.Conv2d(self.c3[0], self.c3[1], kernel_size=5, padding=2), nn.ReLU()
        )
        self.p4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(self.num_inputs, self.c4, kernel_size=1), nn.ReLU(),
        )
        
    def forward(self, X):
#         X = X.view((-1, 1, 96, 96))
        
        op1 = F.relu(self.p1(X))
        op2 = F.relu(self.p2(X))
        op3 = F.relu(self.p3(X))
        op4 = F.relu(self.p4(X))
        
        return torch.cat((op1, op2, op3, op4), dim=1)

In [3]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
)

In [4]:
b2 = nn.Sequential(
    nn.Conv2d(64, 64, kernel_size=1), nn.ReLU(),
    nn.Conv2d(64, 192, kernel_size=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
)

In [5]:
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [6]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [7]:
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten())

net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

In [8]:
X = torch.rand(size=(1, 1, 96, 96))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])
